# Incorporating Phage Biomass
This notebook is adapted from:

Hadrien Delattre, Kalesh Sasidharan, Orkun S Soyer.
Life Science Alliance Nov 2020, 4 (1). DOI: 10.26508/lsa.202000869

https://github.com/OSS-Lab/FBAhv/tree/master

In [25]:
import cobra
from cobra import Reaction
from collections import Counter
from Bio import SeqIO

from cobra.io import read_sbml_model, write_sbml_model

# Constants and Conventions

In [26]:
# Metabolite (verbose to model) definitions
MET_IDS = {
    "atp": "ATP[c]",
    "ctp": "CTP[c]",
    "gtp": "GTP[c]",
    "utp": "UTP[c]",
    "amp": "AMP[c]",
    "cmp": "CMP[c]",
    "gmp": "GMP[c]",
    "ump": "UMP[c]",
    "datp": "dATP[c]",
    "dctp": "dCTP[c]",
    "dgtp": "dGTP[c]",
    "dttp": "dTTP[c]",
    "damp": "dAMP[c]",
    "dcmp": "dCMP[c]",
    "dgmp": "dGMP[c]",
    "dtmp": "dTMP[c]",
    "A": "L_Alanine[c]",
    "R": "L_Arginine[c]",
    "N": "L_Asparagine[c]",
    "D": "L_Aspartate[c]",
    "C": "L_Cysteine[c]",
    "Q": "L_Glutamine[c]",
    "E": "L_Glutamate[c]",
    "G": "Glycine[c]",
    "H": "L_Histidine[c]",
    "I": "L_Isoleucine[c]",
    "L": "L_Leucine[c]",
    "K": "L_Lysine[c]",
    "M": "L_Methionine[c]",
    "F": "L_Phenylalanine[c]",
    "P": "L_Proline[c]",
    "S": "L_Serine[c]",
    "T": "L_Threonine[c]",
    "W": "L_Tryptophan[c]",
    "Y": "L_Tyrosine[c]",
    "V": "L_Valine[c]",
    "h2o": "H2O[c]",
    "adp": "ADP[c]",
    "Pi": "Orthophosphate[c]",
    "h": "H[c]",
    "PPi": "Diphosphate[c]",  # Pyrophosphate
}

# Nucleotides Dictionary in g/mol
NUCLEOTIDE_MASS = {
    "atp": 507.181,
    "gtp": 483.15644,
    "ctp": 523.18062,
    "utp": 484.14116,
    "datp": 491.1816,
    "dgtp": 467.1569,
    "dctp": 482.1683,
    "dttp": 507.181,
    "damp": 331.068,
    "dgmp": 347.063,
    "dcmp": 307.057,
    "dtmp": 322.057,
}
# Amino Acids Dictionary g/mol
AMINO_ACID_MASS = {
    "A": 89.09322,  # Alaline
    "R": 174.201,  # Arginine
    "N": 132.118,  # Asparagine
    "D": 133.1027,  # Aspartate
    "C": 121.158,  # Cysteine
    "Q": 146.14458,  # Glutamine
    "E": 147.1293,  # Glutamate
    "G": 75.06664,  # Glycine
    "H": 155.15468,  # Histidine
    "I": 131.17296,  # Isoleucine
    "L": 131.17296,  # Leucine
    "K": 146.18764,  # Lysine
    "M": 149.21238,  # Methionine
    "F": 165.18918,  # Phenylalanine
    "P": 115.1305,  # Proline
    "S": 105.09262,  # Serine
    "T": 119.1192,  # Threonine
    "W": 204.22526,  # Tryptophan
    "Y": 181.18858,  # Tyrosine
    "V": 117.14638,  # Valine
}
# Misc. Dictionary
MISC_MASS = {
    "PPi": 173.94332,  # Pyrophosphate
}
# Avogadro's Number
N_A = 6.0221409e23
# ATP requirement coefficients
# source: Queka, Dietmaira, Hanschob, Martíneza, Borthb, Nielsen
# Journal of Biotechnology 184 (2014) 172–178
K_ATP_PROTEIN = 4.3
K_ATP_DNA = 1.4
K_ATP_RNA = 0.4

PROTEINS_PER_MRNA = 540

K_PPI = 1  # pyrophosphate production ratio


# Defining Virus Protein Composition

In [27]:
# NOTE: the keys in this dictionary correspond to the id of the product in the genbank file
VIRUS_COMPOSITION = {
    "PHM2": {  # P-HM2                     [Cg]
        "Cg": 1,
        "proteins": {
            "ADO99900.1": 985,
            "ADO99896.1": 12,
            "ADO99878.1": 870,
            "ADO99884.1": 155,
            "ADO99893.1": 5,
            "ADO99885.1": 12,
            "ADO99886.1": 6,
            "ADO99864.1": 24,
            "ADO99942.1": 18,
            "ADO99865.1": 12,
            "ADO99863.1": 12,
            "ADO99793.1": 6,
            "ADO99862.1": 6,
            "ADO99803.1": 3,
            "ADO99913.1": 3,
            "ADO99860.1": 1,
            "ADO99798.1": 1,
            "ADO99895.1": 145,
            "ADO99800.1": 1,
            "ADO99799.1": 1,
            "ADO99874.1": 40,
            "ADO99872.1": 21,
            "ADO99794.1": 6,
            "ADO99894.1": 138,
            "ADO99903.1": 6,
            "ADO99887.1": 6,
            "ADO99947.1": 3,
            "ADO99993.1": 2,
            "ADO99924.1": 1,
            "ADO99795.1": 6,
            "ADO99796.1": 6,
        },
    },
}


# Utility Functions

In [28]:
def multiply_counter(counter, n):
    """return a collection.Counter whose values have been multiplied by a
    scalar"""
    # the multiplicand does not need to be a collections.Counter, but at least
    # it needs to be a dict
    assert isinstance(counter, dict)
    assert float(n)
    return Counter({key: value * n for key, value in counter.items()})


def get_virus_names(virus_record):
    """Return a tuple containing a short name and a
    full name for the virus. This is an alternative to how it is done in Sean's
    genVBOF."""
    long_name = virus_record.description.rstrip(", complete genome")
    # try to get a short name for the virus by taking the prefix of the first CDS
    short_name = (
        next(feature for feature in virus_record.features if feature.type == "CDS")
        .qualifiers["locus_tag"][0]
        .rstrip("_gp01")
    )
    return (short_name, long_name)


def reverse_complement(seq):
    rc = ""
    for nuc in seq[::-1]:
        if nuc == "A":
            rc += "T"
        elif nuc == "T":
            rc += "A"
        elif nuc == "C":
            rc += "G"
        elif nuc == "G":
            rc += "C"
        else:
            raise ValueError("Invalid nucleotide: %s" % nuc)
    return rc

# Main Function

In [29]:
def generate_virus_biomass(virus_record, model, model_name=None):
    """New version of the genVBOF function by Hadrien.
    Builds a Virus Biomass Objective Function (basically a virus biomass
    production reaction, from aminoacids and nucleotides) from a genbank (.gb)
    file. Modified from the original by Jordan C. Rozum and William Sineath.

    Params:
    - virus_record: genbank record of a virus (output from Bio.SeqIO.parse)
    - model: a cobra metabolic model (cobra.core.model.Model)

    Returns:
    - virus biomass objective function (cobra.core.reaction.Reaction)
    """

    # VIRUS IDENTIFICATION
    taxonomy = " ".join(
        [taxon.lower() for taxon in virus_record.annotations["taxonomy"]]
    )
    if "ssp7" not in taxonomy and "hm2" not in taxonomy:
        raise NotImplementedError(
            "Virus family is not supported: Unable to create VBOF. Consult _README"
        )
    short_name, full_name = get_virus_names(virus_record)

    # AMINOACID COUNT
    all_cds = [feature for feature in virus_record.features if feature.type == "CDS"]
    # Check that our own VIRUS_COMPOSITION dict contain exactly the
    # proteins defined in the genbank file, no more, no less.

    protein_ids_in_gb_file = {
        cds.qualifiers["protein_id"][0] for cds in all_cds
    }  # not allowing set to be created
    protein_ids_in_our_data = {
        protein_id for protein_id in VIRUS_COMPOSITION[short_name]["proteins"]
    }
    assert protein_ids_in_our_data.issubset(protein_ids_in_gb_file)
    # make sure that gb file has all the protein ids in our data

    virus_aa_composition = Counter()
    virus_mrna_composition = Counter()
    # protein name -> number of atp involved in its peptide bonds formations
    # (accounting for the number of copies of protein)
    peptide_bond_formation = dict()
    for cds in all_cds:
        if cds.qualifiers["protein_id"][0] in protein_ids_in_our_data:
            protein_id = cds.qualifiers["protein_id"][0]
            aa_sequence = cds.qualifiers["translation"][0]
            dna_sequence = virus_record.seq[cds.location.start : cds.location.end]
            if cds.location.strand == -1:
                dna_sequence = reverse_complement(dna_sequence)
            aa_count = Counter(aa_sequence)
            nc_count = Counter(dna_sequence)
            copies_per_virus = VIRUS_COMPOSITION[short_name]["proteins"][
                protein_id
            ]  ##can change logic to count
            virus_aa_composition += multiply_counter(aa_count, copies_per_virus)
            virus_mrna_composition += multiply_counter(nc_count, copies_per_virus)

            peptide_bond_formation[protein_id] = (
                len(aa_sequence) * K_ATP_PROTEIN - K_ATP_PROTEIN
            ) * copies_per_virus

    # [3] Precursor frequency
    # Genome                            [Nucleotides]
    Cg = VIRUS_COMPOSITION[short_name]["Cg"]  # number of genome copies per virus
    virus_nucl_count = Counter(str(virus_record.seq))
    countA = virus_nucl_count["A"]
    countC = virus_nucl_count["C"]
    countG = virus_nucl_count["G"]
    countT = virus_nucl_count["T"]
    antiA = countT
    antiC = countG
    antiG = countC
    antiT = countA

    # Note: mRNA is 1-stranded, so we don't track the "anti" parts
    # Also: we didn't bother to replace T->U in the string; we do it here
    countA_rna = virus_mrna_composition["A"] / PROTEINS_PER_MRNA
    countC_rna = virus_mrna_composition["C"] / PROTEINS_PER_MRNA
    countG_rna = virus_mrna_composition["G"] / PROTEINS_PER_MRNA
    countU_rna = virus_mrna_composition["T"] / PROTEINS_PER_MRNA

    # Count summation (not used)
    # totNTPS = Cg * (countA + countC + countG + countT + antiA + antiC + antiG + antiT)
    # totAA = sum(count for count in virus_aa_composition.values())

    # [4] VBOF Calculations
    # Nucleotides
    # mol.dntps/mol.virus
    V_a = Cg * (countA + antiA)
    V_c = Cg * (countC + antiC)
    V_g = Cg * (countG + antiG)
    V_t = Cg * (countT + antiT)
    # g.dnmps/mol.virus
    G_a = V_a * NUCLEOTIDE_MASS["damp"]
    G_c = V_c * NUCLEOTIDE_MASS["dcmp"]
    G_g = V_g * NUCLEOTIDE_MASS["dgmp"]
    G_t = V_t * NUCLEOTIDE_MASS["dtmp"]

    # mol.ntps (mrna) / mol.virus
    V_a_rna = countA_rna
    V_c_rna = countC_rna
    V_g_rna = countG_rna
    V_u_rna = countU_rna

    # Amino Acids
    # g.a/mol.virus
    G_aa = {
        aa: count * AMINO_ACID_MASS[aa] for aa, count in virus_aa_composition.items()
    }
    # Molar mass of the virus particles (g.virus/mol.virus); note that the mRNA is not packaged
    M_v = (G_a + G_c + G_g + G_t) + sum(G_aa.values())

    # Stoichiometric coefficients
    # Nucleotides [mmol.ntps/g.virus] (for the genome)
    S_datp = 1000 * (V_a / M_v)
    S_dctp = 1000 * (V_c / M_v)
    S_dgtp = 1000 * (V_g / M_v)
    S_dttp = 1000 * (V_t / M_v)

    # Amino acids [mmol.aa/g.virus]
    S_aa = {aa: 1000 * V_aa / M_v for aa, V_aa in virus_aa_composition.items()}

    # mRNA is handeled differently because the virus doesn't pack it. We will
    # allow it to decay into NMPs at a rate governed by the protein to mrna
    # ratio (for mass conservation)
    S_atp_rna = 1000 * (V_a_rna / M_v)
    S_ctp_rna = 1000 * (V_c_rna / M_v)
    S_gtp_rna = 1000 * (V_g_rna / M_v)
    S_utp_rna = 1000 * (V_u_rna / M_v)
    S_amp_rna = 1000 * (V_a_rna / M_v)
    S_cmp_rna = 1000 * (V_c_rna / M_v)
    S_gmp_rna = 1000 * (V_g_rna / M_v)
    S_ump_rna = 1000 * (V_u_rna / M_v)

    # Energy requirements -(2+n_proteins/PROTEINS_PER_MRNA) is coming from total
    # strands: 2 for DNA, 1 for mRNA This is because we are counting the number
    # of bonds (one less than number of nucleotides in a strand). Not that this
    # term will be exceedingly small compared to the rest, so accuracy here is
    # not critical.
    n_proteins = len(VIRUS_COMPOSITION[short_name]["proteins"])
    S_ppi = (
        S_atp_rna
        + S_ctp_rna
        + S_gtp_rna
        + S_utp_rna
        + S_datp
        + S_dctp
        + S_dgtp
        + S_dttp
        - (2 + n_proteins / PROTEINS_PER_MRNA) * 1000 / M_v
    ) * K_PPI

    # Proteome: Peptide bond formation [ATP + H2O]
    # Note: ATP used in this process is denoated as ATPe/Ae [e = energy version]
    V_Ae = sum(peptide_bond_formation.values())
    S_Ae = 1000 * (V_Ae / M_v)

    # [5] VBOF Reaction formatting and output
    # Left-hand terms: Nucleotides
    S_ATP = S_Ae * -1 - S_atp_rna
    S_CTP = S_ctp_rna * -1
    S_GTP = S_gtp_rna * -1
    S_UTP = S_utp_rna * -1
    S_AMP = S_amp_rna
    S_CMP = S_cmp_rna
    S_GMP = S_gmp_rna
    S_UMP = S_ump_rna
    S_DATP = S_datp * -1
    S_DCTP = S_dctp * -1
    S_DGTP = S_dgtp * -1
    S_DTTP = S_dttp * -1

    # Left-hand terms: Amino Acids
    S_AAf = {aa: -coef for aa, coef in S_aa.items()}
    # Left-hand terms: Energy Requirements
    S_H2O = S_Ae * -1
    # Right-hand terms: Energy Requirements
    S_ADP = S_Ae
    S_Pi = S_Ae
    S_H = S_Ae

    reaction_name = short_name + "_prodrxn_VN"
    virus_reaction = Reaction(reaction_name)
    virus_reaction.name = full_name + " production reaction"
    virus_reaction.subsystem = "Virus Production"
    virus_reaction.lower_bound = 0
    virus_reaction.upper_bound = 1000
    model.add_reactions([virus_reaction])
    virus_reaction.add_metabolites(
        (
            {
                MET_IDS["atp"]: S_ATP,
                MET_IDS["ctp"]: S_CTP,
                MET_IDS["gtp"]: S_GTP,
                MET_IDS["utp"]: S_UTP,
                MET_IDS["amp"]: S_AMP,
                MET_IDS["cmp"]: S_CMP,
                MET_IDS["gmp"]: S_GMP,
                MET_IDS["ump"]: S_UMP,
                MET_IDS["datp"]: S_DATP,
                MET_IDS["dctp"]: S_DCTP,
                MET_IDS["dgtp"]: S_DGTP,
                MET_IDS["dttp"]: S_DTTP,
                MET_IDS["A"]: S_AAf["A"],
                MET_IDS["R"]: S_AAf["R"],
                MET_IDS["N"]: S_AAf["N"],
                MET_IDS["D"]: S_AAf["D"],
                MET_IDS["C"]: S_AAf["C"],
                MET_IDS["Q"]: S_AAf["Q"],
                MET_IDS["E"]: S_AAf["E"],
                MET_IDS["G"]: S_AAf["G"],
                MET_IDS["H"]: S_AAf["H"],
                MET_IDS["I"]: S_AAf["I"],
                MET_IDS["L"]: S_AAf["L"],
                MET_IDS["K"]: S_AAf["K"],
                MET_IDS["M"]: S_AAf["M"],
                MET_IDS["F"]: S_AAf["F"],
                MET_IDS["P"]: S_AAf["P"],
                MET_IDS["S"]: S_AAf["S"],
                MET_IDS["T"]: S_AAf["T"],
                MET_IDS["W"]: S_AAf["W"],
                MET_IDS["Y"]: S_AAf["Y"],
                MET_IDS["V"]: S_AAf["V"],
                MET_IDS["h2o"]: S_H2O,
                MET_IDS["adp"]: S_ADP,
                MET_IDS["Pi"]: S_Pi,
                MET_IDS["h"]: S_H,
                MET_IDS["PPi"]: S_ppi,
            }
        )
    )
    return virus_reaction

# Initial modifications to the model

Here, alter the DNA and RNA formation reactions so that they are built from (d)NTP and produce pyrophosphate.

We print a before and after comparison of the optimal biomass. Note that it does not significantly change.

In [30]:
virus_record = SeqIO.read("./model_files/hm2.gbff", "genbank")
model = read_sbml_model("./model_files/ProchlorococcusMED4v1.xml")


print("BEFORE:")
print(f"BIOMASS = {model.slim_optimize()}")

m1 = model.reactions.BDNA.metabolites
nt = 0
for m, s in m1.items():
    if m.id in ["dATP[c]", "dCTP[c]", "dGTP[c]", "dTTP[c]"]:
        nt += s
# Note: we are neglecting the number of strands, but this will have a neglible
# effect since this will be much smaller than the number of nucleotides

m1[model.metabolites.get_by_id("Diphosphate[c]")] = -nt
model.reactions.BDNA.add_metabolites(m1)

m2 = {}
nt = 0
original = dict(model.reactions.BRNA.metabolites.items())
for m, s in model.reactions.BRNA.metabolites.items():
    if m.id in ["AMP[c]", "CMP[c]", "GMP[c]", "UMP[c]"]:
        nt += s
        ntp = m.id.replace("M", "T")
        if ntp == "ATP[c]":
            met = model.metabolites.get_by_id(ntp)
            m2[met] = s + model.reactions.BRNA.metabolites[met]
        else:
            m2[model.metabolites.get_by_id(ntp)] = s
    elif m.id != "ATP[c]":
        m2[model.metabolites.get_by_id(m.id)] = s

m2[model.metabolites.get_by_id("Diphosphate[c]")] = -nt

model.reactions.BRNA.subtract_metabolites(original)
model.reactions.BRNA.add_metabolites(m2)

print("AFTER:")
print(f"BIOMASS = {model.slim_optimize()}")

BEFORE:
BIOMASS = 0.09846248800237327
AFTER:
BIOMASS = 0.09846248800237146


### Show the new DNA/RNA reactions:

In [31]:
print(model.reactions.BDNA)
print(model.reactions.BRNA)

BDNA: 22.44 ATP[c] + 22.44 H2O[c] + 1.42514 dATP[c] + 0.64218 dCTP[c] + 0.64218 dGTP[c] + 1.42514 dTTP[c] --> 22.44 ADP[c] + 2.06732 Diphosphate[c] + 22.44 Orthophosphate[c] + 2.0 deoxyribonucleic_acids[c]
BRNA: 8.23 ATP[c] + 0.61 CTP[c] + 0.89 GTP[c] + 7.44 H2O[c] + 0.81 UTP[c] --> 7.44 ADP[c] + 3.1 Diphosphate[c] + 7.44 Orthophosphate[c] + ribonucleic_acids[c]


# Generate the Virus biomass reaction

We print the resulting biomass stoichiomtery, and also output the range of optimal biomass. Note that optimal phage growth precludes host growth.

The resulting model is written to file, with host biomass as the default objective.

In [32]:
generate_virus_biomass(virus_record=virus_record, model=model)
print(" ")
for m, s in sorted(
    model.reactions.PHM2_prodrxn_VN.metabolites.items(), key=lambda x: x[0].id
):
    print(f"{s:+07.3f} x {m.id}")
model.objective = model.reactions.PHM2_prodrxn_VN
fva = cobra.flux_analysis.flux_variability_analysis(
    model, [model.reactions.PHM2_prodrxn_VN, model.reactions.BIOMASS]
)
print(fva)

# reset the objective before writing to file
model.objective = model.reactions.BIOMASS

write_sbml_model(model, "model_files/ProchlorococcusMED4+PHM2v1.xml")

 
+20.446 x ADP[c]
+00.009 x AMP[c]
-20.455 x ATP[c]
+00.005 x CMP[c]
-00.005 x CTP[c]
+01.249 x Diphosphate[c]
+00.006 x GMP[c]
-00.006 x GTP[c]
-00.422 x Glycine[c]
-20.446 x H2O[c]
+20.446 x H[c]
-00.428 x L_Alanine[c]
-00.195 x L_Arginine[c]
-00.312 x L_Asparagine[c]
-00.300 x L_Aspartate[c]
-00.023 x L_Cysteine[c]
-00.247 x L_Glutamate[c]
-00.180 x L_Glutamine[c]
-00.050 x L_Histidine[c]
-00.269 x L_Isoleucine[c]
-00.310 x L_Leucine[c]
-00.192 x L_Lysine[c]
-00.090 x L_Methionine[c]
-00.185 x L_Phenylalanine[c]
-00.164 x L_Proline[c]
-00.389 x L_Serine[c]
-00.451 x L_Threonine[c]
-00.039 x L_Tryptophan[c]
-00.196 x L_Tyrosine[c]
-00.321 x L_Valine[c]
+20.446 x Orthophosphate[c]
+00.007 x UMP[c]
-00.007 x UTP[c]
-00.378 x dATP[c]
-00.233 x dCTP[c]
-00.233 x dGTP[c]
-00.378 x dTTP[c]
                  minimum       maximum
PHM2_prodrxn_VN  0.150551  1.505510e-01
BIOMASS          0.000000 -1.679873e-14


# Explore mass composition

We now examine the DNA/Protein mass ratio for the phage, as well as the total mass of other (unpacked) components required for phage assembly, such as ATP.

In [33]:
# reverse the id strings
met_id_lookup = {v: k for k, v in MET_IDS.items()}

dna_mass = 0
protein_mass = 0
rna_mass = 0
for m, s in sorted(
    model.reactions.PHM2_prodrxn_VN.metabolites.items(), key=lambda x: x[0].id
):
    if s > 0:
        continue  # only looking at consumption right now
    if m.id in ["dATP[c]", "dCTP[c]", "dGTP[c]", "dTTP[c]"]:
        mp = m.id[0:2] + "M" + m.id[3:]
        lookup = met_id_lookup[mp]
        dna_mass -= NUCLEOTIDE_MASS[lookup] * s / 1000
    elif m.id in ["ATP[c]", "CTP[c]", "GTP[c]", "UTP[c]"]:
        lookup = met_id_lookup[m.id]
        if m.id == "ATP[c]":  # for ATP, we separate out energetic and RNA requirements
            sd = model.reactions.PHM2_prodrxn_VN.metabolites[
                model.metabolites.get_by_id("H2O[c]")
            ]
            rna_mass -= NUCLEOTIDE_MASS[lookup] * (s - sd) / 1000
            # other_masses -= NUCLEOTIDE_MASS[lookup] * sd / 1000
        else:
            rna_mass -= NUCLEOTIDE_MASS[lookup] * s / 1000
    elif m.id.startswith("L_") or m.id == "Glycine[c]":
        lookup = met_id_lookup[m.id]
        protein_mass -= AMINO_ACID_MASS[lookup] * s / 1000

print(f"{dna_mass=}, {protein_mass=}, {rna_mass=}")

dna_mass=0.3992977448914946, protein_mass=0.6007022551085055, rna_mass=0.013223616993168272
